In [269]:
import pandas as pd
import numpy as np
import random

def generate_item_df(num_items = 20, weight_range = (0, 10), value_range = (0, 1)):
    weights = np.random.uniform(*weight_range, num_items)
    values = np.random.uniform(*value_range, num_items)
    return pd.DataFrame({
        'Name': [str(i) for i in range(num_items)],
        'Weight': weights,
        'Value': values
    })

df = generate_item_df()
print(df.head(4))

  Name    Weight     Value
0    0  7.224940  0.115832
1    1  5.612525  0.511764
2    2  2.073984  0.220562
3    3  8.391002  0.608882


In [270]:
def genetic_algorithm(df, max_weight=30, population_size=100, num_generations=50, mutation_rate=1):
    weights = df["Weight"].tolist()
    values = df["Value"].tolist()
    n = len(values)
    
    # Normalise mutations, so that longer lists dont become more mutated.
    mutation_rate /= n

    average_weight = sum(weights) / n
    expected_items = max_weight / average_weight
    bit_flip_probability = expected_items / n

    # Initialise the population so that each individual has roughly the correct weight
    population = [[1 if random.random() < bit_flip_probability else 0 for _ in range(n)]
                  for _ in range(population_size)]
    
    def get_total_value(individual):
        return sum(value if individual[i] else 0 for i, value in enumerate(values))
    
    def get_total_weight(individual):
        return sum(weight if individual[i] else 0 for i, weight in enumerate(weights))

    def fitness(individual):
        total_weight = get_total_weight(individual)
        total_value = get_total_value(individual)
        if total_weight > max_weight:
            return 0
        return total_value

    def tournament_selection(population, k=3):
        return max(random.sample(population, k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]

    def mutate(individual):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i] # Flip bit
        return individual

    for _ in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population

    best = max(population, key=fitness)
    return df[[bool(bit) for bit in best]].copy(), get_total_value(best), get_total_weight(best)


In [271]:
def show_genetic_algorithm(df, ga_func=genetic_algorithm, n_runs=4,
                           population_size=100, num_generations=50, mutation_rate=0.2):
    for _ in range(n_runs):
        items, value, weight = ga_func(df, population_size=population_size, num_generations=num_generations, mutation_rate=mutation_rate)
        print(f"Items: {', '.join(items['Name'].tolist())}")
        print(f"Total weight: {weight:.2f}")
        print(f"Total value: {value:.2f}")
        print()


In [272]:
# Test with the original 20 items
show_genetic_algorithm(df)
%timeit genetic_algorithm(df)

Items: 3, 4, 6, 13, 14, 18
Total weight: 29.21
Total value: 4.82

Items: 2, 4, 6, 7, 11, 13, 14, 18
Total weight: 29.82
Total value: 4.99

Items: 4, 6, 7, 13, 14, 18, 19
Total weight: 29.94
Total value: 5.15

Items: 1, 2, 4, 6, 7, 13, 14, 19
Total weight: 28.51
Total value: 4.91

72.8 ms ± 4.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [273]:
# Test with 1000 items - slow
df_1000 = generate_item_df(1000)
show_genetic_algorithm(df_1000)
timeit_1000 = %timeit -o genetic_algorithm(df_1000)

Items: 32, 153, 194, 205, 238, 256, 271, 298, 338, 359, 400, 461, 493, 566, 571, 574, 584, 638, 768, 783, 806, 874, 882, 911, 920, 984
Total weight: 29.89
Total value: 19.75

Items: 33, 71, 133, 153, 158, 193, 200, 262, 333, 397, 400, 426, 461, 655, 664, 669, 703, 734, 760, 783, 804, 874, 890, 935, 952, 984
Total weight: 29.84
Total value: 17.33

Items: 25, 66, 175, 194, 200, 284, 285, 305, 333, 346, 355, 419, 446, 482, 550, 646, 683, 703, 769, 795, 806, 821, 952, 994
Total weight: 29.95
Total value: 17.08

Items: 20, 82, 194, 218, 223, 238, 291, 298, 391, 461, 465, 485, 540, 584, 704, 765, 778, 832, 882, 973
Total weight: 30.00
Total value: 14.03

2.14 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [274]:
# Rewrite using numpy
def genetic_algorithm_np(df, max_weight=30, population_size=100, num_generations=50, mutation_rate=1):
    weights = df["Weight"].to_numpy()
    values = df["Value"].to_numpy()
    n = len(values)
    
    # Normalise mutations, so that longer lists dont become more mutated.
    mutation_rate /= n

    average_weight = np.mean(weights)
    expected_items = max_weight / average_weight
    bit_flip_probability = expected_items / n

    # Initialise the population so that each individual has roughly the correct weight
    population = np.random.rand(population_size, n) < bit_flip_probability
    
    def fitness(individual):
        total_weight = np.dot(weights, individual)
        total_value = np.dot(values, individual)
        return total_value if total_weight <= max_weight else 0

    def tournament_selection(population, k=3):
        return max(random.sample(list(population), k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, n - 1)
        offspring1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        offspring2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        return offspring1, offspring2

    def mutate(individual):
        mutation_mask = np.random.rand(n) < mutation_rate
        individual[mutation_mask] = 1 - individual[mutation_mask] # Flip bit
        return individual

    for _ in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = np.array(new_population)

    best = max(population, key=fitness)
    best_items = df.iloc[best.astype(bool)].copy()
    return best_items, np.dot(values, best), np.dot(weights, best)

In [275]:
# Test numpy version - 10x faster
show_genetic_algorithm(df_1000, ga_func=genetic_algorithm_np)
timeit_1000_fast = %timeit -o genetic_algorithm_np(df_1000)
print(f"{timeit_1000.best/timeit_1000_fast.best:.2f}x faster")

Items: 25, 31, 80, 153, 194, 220, 234, 260, 355, 378, 438, 461, 550, 578, 646, 664, 765, 806, 856, 935, 952, 963
Total weight: 29.90
Total value: 17.72

Items: 49, 74, 200, 205, 223, 252, 310, 461, 463, 487, 646, 699, 708, 756, 765, 805, 806, 808, 829, 863, 935, 975
Total weight: 29.89
Total value: 16.10

Items: 25, 143, 154, 155, 175, 252, 256, 284, 333, 374, 397, 479, 504, 583, 605, 618, 655, 664, 784, 789, 821, 882, 890, 903, 959
Total weight: 29.97
Total value: 16.59

Items: 71, 153, 176, 179, 218, 238, 251, 270, 374, 498, 500, 560, 638, 696, 741, 843, 856
Total weight: 29.98
Total value: 12.11

232 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.25x faster


In [276]:
# Try different mutation rates
n_runs = 10
for mutation_rate in (0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100):
    _, values, weights = list(zip(*[genetic_algorithm_np(df_1000, mutation_rate=mutation_rate) for _ in range(n_runs)]))
    print(f"Mutation rate: {mutation_rate}".ljust(22) + 
        f"Max weight: {max(weights):.3f}  "
        f"Average value: {sum(values)/n_runs:.3f}")

Mutation rate: 0      Max weight: 29.925  Average value: 9.816
Mutation rate: 0.001  Max weight: 29.753  Average value: 10.615
Mutation rate: 0.005  Max weight: 29.952  Average value: 10.677
Mutation rate: 0.01   Max weight: 29.956  Average value: 11.390
Mutation rate: 0.05   Max weight: 30.000  Average value: 14.356
Mutation rate: 0.1    Max weight: 29.995  Average value: 14.532
Mutation rate: 0.5    Max weight: 29.870  Average value: 19.010
Mutation rate: 1      Max weight: 88.288  Average value: 18.102
Mutation rate: 5      Max weight: 1086.124  Average value: 96.895
Mutation rate: 10     Max weight: 1749.592  Average value: 156.361
Mutation rate: 50     Max weight: 2556.457  Average value: 240.127
Mutation rate: 100    Max weight: 2603.142  Average value: 239.680


- It looks like somewhere between the mutation rates of 1 and 5, every member of the population becomes overweight.
- 0.5 - 1 looks optimal (for n = 1000 at least).